# What is about ?

LLE - sklearn.manifold.LocallyLinearEmbedding - one out of many dimensional reduction algorithms used to visualize data ( tsne, umap - other popular tools).

**Finding Briefly.** For MoA dataset LLE produce some visually clearly seen clusters - three "strips" of lines - which clearly correlate with targets and other features.

**Challenge.** At the moment it looks mysterious - can you somehow explain it and possibly extract some benefits for prediction and more generally for  biological research.


**Context** Doing dimensional reduction is stadard for such tasks, in particular some bioinformatitions sometimes prefer LLE because it sometimes produces something like "trajectories" (or line-segments). Sometimes they are meaningless - because LLE tends to produce strainght-line looking segments even when there is not such pattern in data, but not in that case. 


-1) Pay attention - LLE has been performed on data WITHOUT CP_TIME, CP_DOSE, CP_CTL - only genes and viabilities

0) Before reading look at picture below - you will see three "strips"

1) The righest strip contains only elements with cp_time = 24 (pay attention: despite cp_time was deleted) 

2) TWO right strips have all targets = 0 for almost all points (!) (targets were NOT in data) 

3) The control group - placed in the left "strip" almost perfectly  (pay attention: despite control group flag was deleted from the data)

It seem more things can be found. Please try.


PS

I have tried to use these LLE as features for my LogReg models - but they did not improve my scores.

Other dimensional reduction plots can be found at my notebook:
https://www.kaggle.com/alexandervc/moa-data-visualization-via-dimensional-reduct
(LLE plot is also there, but now I decided to attract some attention to it, since it might be valueable not only for competition itself, but for further biological analysis). 

PSPS

Nice patterns are somewhat fragile - using slighly other settings for preprocessing - they might disapper.
Preprocessing setting was found ad hoc. 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load data

In [ ]:
import time 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',index_col = 0)  
df

In [ ]:
y = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv',index_col = 0 )
y

In [ ]:
mode_which_part_to_process = 'full'
if mode_which_part_to_process == 'full':
    # consider only gene expression part 
    X = df[[c for c in df.columns if ('c-' in c) or ('g-' in c)]].values
if mode_which_part_to_process == 'genes':
    # consider only gene expression part 
    X = df[[c for c in df.columns if 'g-' in c]].values
if mode_which_part_to_process == 'c':
    # consider only gene expression part 
    X = df[[c for c in df.columns if 'c-' in c]].values

X_original_save = X.copy()
print(X.shape)

y_sum = y.sum(axis = 1)
df['y_sum']=y_sum
print(y_sum.shape)
y_sum.value_counts()



In [ ]:
X.shape, df.shape, df.columns[-1]

# LLE train LLE with ad hoc found params

1) only first 5000 elements of train taken  (just taken  because otherwise it might run too long)

2) first do pca to 50 dims (standard trick in similar tasks, otherwise it might run too long)

3) random_state = 1 (for  other it also works, but not for all)

In [ ]:
from sklearn import manifold
import time 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition  import PCA

pca = PCA()
# Preliminary reduce X to speed up:
r = pca.fit_transform(X.copy())
i_cut = 5000
r = r[:i_cut,:50]


n_neighbors = 10
n_components = 2
method = manifold.LocallyLinearEmbedding(n_neighbors=n_neighbors, n_components = n_components, 
                                         eigen_solver='auto', method='standard', random_state=1)# , n_jobs=-1)

t0 = time.time()
r = method.fit_transform(r)
t1 = time.time()
print("%s: %.2g sec" % ('LLE', t1 - t0))

fig = plt.figure(figsize = (15,7) )
f = 'cp_time'
c_tmp = df[f].copy().apply(lambda x: str(x)+'_' )# convert to string to avoid sns.scatterplot to interpolate numeric values in legend
sns.scatterplot(x=r[:,0], y=r[:,1] , hue =c_tmp[:i_cut]) #  df[f][:i_cut].apply(lambda x: str(x) )  )
#plt.scatter(r[:,0], y=r[:,1] , c=df[f][:i_cut]  )
#plt.legend()
plt.title('Colored by '+f)
plt.show()


# Apply trained transforms to full dataset - again see - rightest "strip" is mostly by cp_time =24 (despite cp_time was deleted from X)


In [ ]:
# Preliminary reduce X to speed up:
r = pca.transform(X.copy())

t0 = time.time()
r = method.transform(r[:,:50])
t1 = time.time()
print("%s: %.2g sec" % ('LLE', t1 - t0))

fig = plt.figure(figsize = (15,7) )
f = 'cp_time'
c_tmp = df[f].copy().apply(lambda x: str(x)+'_' )# convert to string to avoid sns.scatterplot to interpolate numeric values in legend
sns.scatterplot(x=r[:,0], y=r[:,1] , hue =c_tmp) #  df[f][:i_cut].apply(lambda x: str(x) )  )
#plt.scatter(r[:,0], y=r[:,1] , c=df[f][:i_cut]  )
#plt.legend()
plt.title('Colored by '+f)
plt.show()


# Color by target sum and other - observe on control group and targets patterns - see details below

In [ ]:
# Apply transform to full dataset 

r = pca.transform(X.copy() )
r = r[:,:50]

t0 = time.time()
r = method.transform(r)
print(time.time()-t0,'seconds passed')

fig = plt.figure(figsize = (15,7) )
c = 0
for f in ['cp_dose', 'cp_type','cp_time', 'y_sum']:
    c+=1; fig.add_subplot(1, 4 , c) 
    c_tmp = df[f].copy().apply(lambda x: str(x)+'_' )# convert to string to avoid sns.scatterplot to interpolate numeric values in legend
    sns.scatterplot(x=r[:,0], y=r[:,1] , hue = c_tmp  )
    plt.title('Colored by '+f)
plt.show()


# Look at control group - almost all elements - in the leftest "strip"

In [ ]:
df['cp_type'].value_counts()

In [ ]:
# Apply transform to full dataset 

r = pca.transform(X.copy() )
r = r[:,:50]

t0 = time.time()
r = method.transform(r)
print(time.time()-t0,'seconds passed')

fig = plt.figure(figsize = (15,7) )
c = 0
for f in ['cp_dose', 'cp_type','cp_time', 'y_sum']:
    c+=1; fig.add_subplot(1, 4 , c) 
    m = df['cp_type'] == 'ctl_vehicle'
    c_tmp = df[f].copy().apply(lambda x: str(x)+'_' )# convert to string to avoid sns.scatterplot to interpolate numeric values in legend
    sns.scatterplot(x=r[:,0][m], y=r[:,1][m] , hue = c_tmp[m]  )
    plt.title('Colored by '+f)
plt.show()


# Two right strips - MoA = ZERO, almost perfectly

In [ ]:
# Apply transform to full dataset 

r = pca.transform(X.copy() )
r = r[:,:50]

t0 = time.time()
r = method.transform(r)
print(time.time()-t0,'seconds passed')

fig = plt.figure(figsize = (15,7) )
c = 0
for f in ['cp_dose', 'cp_type','cp_time', 'y_sum']:
    c+=1; fig.add_subplot(1, 4 , c) 
    m = df['y_sum'] > 0 # dfy_sum'] == 0 # cp_type'] == 'trt_cp' #'ctl_vehicle'
    c_tmp = df[f].copy().apply(lambda x: str(x)+'_' )# convert to string to avoid sns.scatterplot to interpolate numeric values in legend
    sns.scatterplot(x=r[:,0][m], y=r[:,1][m] , hue = c_tmp[m]  )
    plt.title('Colored by '+f)
plt.show()


In [ ]:
# More things should be checked - please try